# 接雨水

## 問題
给定 n 个非负整数表示每个宽度为 1 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。

示例 1：

<img src= 'https://code-thinking-1253855093.cos.ap-guangzhou.myqcloud.com/pics/20210713205038.png'>

输入：height = [0,1,0,2,1,0,1,3,2,1,2,1]
输出：6
解释：上面是由数组 [0,1,0,2,1,0,1,3,2,1,2,1] 表示的高度图，在这种情况下，可以接 6 个单位的雨水（蓝色部分表示雨水）。


## 代碼


本文深度讲解如下三种方法：

- 双指针法
- 动态规划
- 单调栈


### 雙指針
按列來計算
<img src = 'https://img-blog.csdnimg.cn/20210402091208445.png'>

首先从头遍历所有的列，并且要注意第一个柱子和最后一个柱子不接雨水，代码如下：



In [ ]:
class Solution(object):
    def trap(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        res = 0
        
        for i in range(len(height)):
            # 第一個柱子和最後一個柱子不接雨水
            if i==0 or i==len(height)-1:
                continue
            lHeight = height[i-1]
            rHeight = height[i+1]

            for r in (i+2,len(height)):
                if height[r]>rHeight:
                    rHeight = height[r]
            for l in (i-1):
                if height[l]>lHeight:
                    lHeight = height[l]
            H = min(lHeight,rHeight)-height[i]
            if H>0:
                res += H 
        return res 


### 動態規劃

为了得到两边的最高高度，使用了双指针来遍历，每到一个柱子都向两边遍历一遍，这其实是有重复计算的。我们把每一个位置的左边最高高度记录在一个数组上（maxLeft），右边最高高度记录在一个数组上（maxRight）。这样就避免了重复计算，这就用到了动态规划。

当前位置，左边的最高高度是前一个位置的左边最高高度和本高度的最大值。

即从左向右遍历：**maxLeft[i] = max(height[i], maxLeft[i - 1]);**

从右向左遍历：**maxRight[i] = max(height[i], maxRight[i + 1]);**

In [ ]:
class Solution(object):
    def trap(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        if len(height)<=2:
            return 0 
        
        maxLeft = [0]*len(height)
        maxRight = [0]*len(height)
        
        size = len(maxRight)

        # 紀錄每個柱子左邊柱子最大高度
        maxLeft[0] = height[0]
        for i in range(1,size):
            maxLeft[i] = max(height[i],maxLeft[i-1])
        # 紀錄每個柱子右邊柱子的最大高度
        maxRight[size-1]=height[size-1]
        for i in range(size-2,-1,-1):
            maxRight[i] = max(height[i],maxRight[i+1])
        
        # 求和
        sum = 0 
        for i in range(0,size):
            count = min(maxLeft[i],maxRight[i])-height[i]
            if count>0:
                sum += count
        return sum 


### 單調輚
单调栈是按照 行 的方向来计算雨水

从栈顶到栈底的顺序：从小到大

通过三个元素来接水：栈顶，栈顶的下一个元素，以及即将入栈的元素

雨水高度是 min(凹槽左边高度, 凹槽右边高度) - 凹槽底部高度

雨水的宽度是 凹槽右边的下标 - 凹槽左边的下标 - 1（因为只求中间宽度）

In [ ]:
class Solution(object):
    def trap(self, height):
        """
        :type height: List[int]
        :rtype: int
        
        """
        # stack存儲index，用於計算相應的柱子高度
        stack = [0]
        result = 0
        for i in range(1,len(height)):
            # 情況1
            if height[i]<height[stack[-1]]:
                stack.append(i)
            # 情況2
            # 当当前柱子高度和栈顶一致时，左边的一个是不可能存放雨水的，所以保留右侧新柱子
            # 需要使用最右边的柱子来计算宽度
            elif height[i]==height[stack[-1]]:
                stack.pop()
                stack.append(i)
            # 情況3
            else:
                # 拋出所有較低的柱子
                while stack and height[i]>height[stack[-1]]:
                    ## 栈顶就是中间的柱子：储水槽，就是凹槽的地步
                    mid_height = height[stack[-1]]
                    stack.pop()
                    if stack:
                        right_height = height[i]
                        left_height = height[stack[-1]]
                        # 两侧的较矮一方的高度 - 凹槽底部高度
                        h = min(right_height, left_height) - mid_height
                        # 凹槽右侧下标 - 凹槽左侧下标 - 1: 只求中间宽度
                        w = i - stack[-1] - 1
                        # 体积：高乘宽
                        result += h * w
                stack.append(i)
        return result
        
        